In [9]:
import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [14]:
Root = os.getcwd()
os.makedirs('Images_Folder', exist_ok=True)
os.makedirs('Result_Folder', exist_ok=True)

In [15]:
def Convert_to_PNG():
    os.chdir(os.path.join(Root,'Images_Folder'))
    image_patterns = ["*.jpeg", "*.jpg"]
    for pattern in image_patterns:
        for image_file in glob.glob(pattern):
            if not image_file.endswith(".png"):
                new_file = os.path.splitext(image_file)[0] + ".png"
                os.rename(image_file, new_file)
    os.chdir(Root)

In [16]:
def Image_Show(Image):
    plt.imshow(Image, cmap='gray')
    plt.axis('off')
    plt.show() 

In [17]:
def Convert_Image_To_Matrix(Image_Name):
    if Image_Name in os.listdir('Images_Folder'):
        Image = cv2.imread(os.path.join('Images_Folder', Image_Name))
        Gray_Image = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
        Data = np.array(Gray_Image)/255 # Chuẩn hóa dữ liệu
        Image_Show(Gray_Image)
        return Data
    else: print("Picture not found!!!!")